<a href="https://colab.research.google.com/github/khajum/ai-playground/blob/main/rag/rag-application-101/pdf_load_to_chromaDB_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Pipelines - Data Ingestion to Verctor DB Pipeline

In [ ]:
# Setup: After required folder is created by below script, upload sample pdf files

# Create folder data/pdf under current directory
os.makedirs("./data/pdf", exist_ok=True)

In [ ]:
# Install pypdf if not already installed
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install pypdf
!pip install pymupdf

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
from pathlib import Path

# Read all the PDFs inside directory './data/pdf/'
def process_all_pdfs(pdf_directory):
  # Process all the pdf files inside the directory
  all_documents = []
  pdf_dir = Path(pdf_directory)

  # Find all the pdf files recursively
  pdf_files = list(pdf_dir.glob('**/*.pdf'))

  # print all the pdf files found in the directory
  print(f"Found {len(list(pdf_files))} pdf files in {pdf_directory}")

  for pdf_file in pdf_files:
    print(f"\nProcessing: {pdf_file.name}")
    try:
      loader = PyPDFLoader(str(pdf_file))
      documents = loader.load()

      # Add source information to metadata
      for doc in documents:
        doc.metadata['source_file'] = str(pdf_file.name)
        doc.metadata['file_type'] = "pdf"

      all_documents.extend(documents)
      print(f" Processed {pdf_file.name} with {len(documents)} pages")
    except Exception as e:
      print(f" Error processing {pdf_file.name}: {e}")

  print(f" Total documents processed: {len(all_documents)}")
  return all_documents

# Process all the PDFs in the data directory
# Call the corrected function
all_pdf_document = process_all_pdfs("./data")
print(f"Successfully processed and collected {len(all_pdf_document)} documents.")


In [ ]:
!pip install langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_document(documents, chunk_size=1000, chunk_overlap=200):
  # split the documents into smaller chucks for better RAG performance
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap,
      length_function=len,
      separators = ["\n", "\n\n", " ", ""])
  splits = text_splitter.split_documents(documents)
  return splits

In [ ]:
# Install all embeding and vectordb related packaged if not already installed
!pip install sentence-transformers faiss-cpu chromadb

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager:
  """Handles the document embeding generation using SentenceTransformer"""

  def __init__(self, embedding_model_name:str="all-MiniLM-L6-v2"):
    """
    Initialize the EmbeddingManager.

    Args:
      embedding_model_name (str): Name of the embedding model to use.
    """
    self.embedding_model_name = embedding_model_name
    self.model = None
    self._load_model()

  def _load_model(self):
    """
    Load the SentenceTransformer embedding model.
    """
    try:
      print(f"Loading SentenceTransformer model: {self.embedding_model_name}")
      self.model = SentenceTransformer(self.embedding_model_name)
      print(f"Loaded SentenceTransformer model successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
    except Exception as e:
      print(f"Error loading embedding model: {e}")
      raise

  def generate_embeddings(self, texts:List[str]) -> np.ndarray:
    """
    Generate embeddings for a list of texts.

    Args:
      texts (List[str]): List of texts to generate embeddings for.

    Returns:
      np.ndarray: Array of embeddings.
    """
    if self.model is None:
      self._load_model()
    try:
      print(f"Generating embeddings for {len(texts)} texts...")
      embeddings = self.model.encode(texts, show_progress_bar=True)
      print(f"Successfully generated embeddings for {len(texts)} texts with shape: {embeddings.shape}")
      return embeddings
    except Exception as e:
      print(f"Error generating embeddings: {e}")
      raise


# Initialize the embedding manager
embedding_manager = EmbeddingManager()
embeddings = embedding_manager.generate_embeddings([doc.page_content for doc in all_pdf_document])


In [ ]:
!pip install chromadb

In [ ]:
from pathlib import PureWindowsPath
from typing import Self, Any
import os
import chromadb
import numpy as np

class ChromaVectorStore:
  """ Manages document embeddings into a ChromaDB vector store"""
  def __init__(self, collection_name="pdf_documents", persist_directory="./data/vector_store"):
    """
    Initialize the vector store

    Arg:
      collection_name: Name of ChromaDB collection
      persist_directory: Path to store ChromaDB data
    """
    self.collection_name = collection_name
    self.persist_directory = persist_directory
    self._initialize_store()

  def _initialize_store(self):
    """ Initialize the ChromaDB client and collection"""
    try:
      # Create persistent ChromaDB client
      os.makedirs(self.persist_directory, exist_ok=True)
      self.client = chromadb.PersistentClient(path=self.persist_directory)

      # Get or create collection
      self.collection = self.client.get_or_create_collection(
          name=self.collection_name,
          metadata={"description":"PDF document embeddings for RAG"})

      print(f"ChromaDB Vector Store initialized with collection: '{self.collection_name}' at path: '{self.persist_directory}'")
      print(f"Existing documents in the collection: {self.collection.count()}")

    except Exception as e:
      print(f"Error initializing ChromaDB vector store db: {e}")
      raise

  def add_documents(self, documents: list[Any], embeddings: np.ndarray):
    """ Add documents and their embeddings to the vector store

    Args:
      documents: List of documents to add
      embeddings: List of embeddings for the documents
    """
    if len(documents) != len(embeddings):
      raise ValueError("Number of documents and embeddings must be the same")

    print(f"Adding {len(documents)} documents to the vector store")

    # Prepare data for Chroma DB
    ids = []
    metadatas = []
    documents_text = []
    embeddings_list = []

    for i, (document, embedding) in enumerate(zip(documents, embeddings)):
      # generate unique Id
      document_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
      ids.append(document_id)

      # prepare metadata
      metadata = dict(document.metadata)
      metadata['doc_index'] = i
      metadata['content_length'] = len(document.page_content)
      metadatas.append(metadata)

      # prepare document text
      documents_text.append(document.page_content)

      # Embedding
      embeddings_list.append(embedding.tolist())

    # Add data to Chroma DB
    self.collection.add(
        ids=ids,
        metadatas=metadatas,
        documents=documents_text,
        embeddings=embeddings_list
    )
    print(f"Added {len(documents)} documents to the vector store")
    print(f"Total documents in the collection: {self.collection.count()}")


vector_store = ChromaVectorStore()

In [ ]:
split_chunks = split_document(all_pdf_document);
print(split_chunks)

In [ ]:
# Convert the text to embeddings
texts = [doc.page_content for doc in split_chunks]
# generate embeddings
embeddings = embedding_manager.generate_embeddings(texts)

# store in the vecotor DB
vector_store.add_documents(split_chunks, embeddings)


In [ ]:
class RAGRetreiver:
  """ handles query-based retrieval from the vector store"""

  def __init__(self, vector_store: ChromaVectorStore, embedding_manager: EmbeddingManager):
    """
    Initialize the RAG retriever

    Args:
      vector_store: ChromaDB vector store
      embedding_manager: Embedding manager
    """
    self.vector_store = vector_store
    self.embedding_manager = embedding_manager

  def retrieve(self, query: str, top_k: int = 5, score_threshold: float=0.0) -> List[Dict[str, Any]]:
    """
    Retrieve documents based on a query

    Args:
      query: Query string
      top_k: Number of documents to retrieve

    Returns:
      List of retrieved documents
    """
    print(f"Retrieving documents for query: '{query}'")
    print(f"Top K: {top_k}, Score Threshold: {score_threshold}")
    try:
      # Generate embeddings for the query
      query_embedding = self.embedding_manager.generate_embeddings([query])
      #print(f"Query embedding generated: {query_embedding}")

      # search in vector store
      print(f"Searching for documents in vector store...")
      results = self.vector_store.collection.query(
          query_embeddings=query_embedding.tolist(),
          n_results=top_k

      )
      print(f"Found {len(results['documents'])} documents in the vector store")
      #print(f"Results: {results}")

      #process results
      retrieved_documents = []

      for i, document in enumerate(results['documents'][0]):

        score = results['distances'][0][i]
        if score < score_threshold:
          continue
        metadata = results['metadatas'][0][i]
        ids = results['ids'][0][i]

        retrieved_documents.append({
            'id': ids,
            'page_content': document,
            'metadata': metadata,
            'similarity_score': 1 - score,
            'distance': score,
            'rank': i + 1
        })
      print(f"Successfully retrieved {len(retrieved_documents)} documents")
      return retrieved_documents

    except Exception as e:
      print(f"Error retrieving documents: {e}")
      raise

In [ ]:
# test RAG Retreiver for a given query
rag_retriever = RAGRetreiver(vector_store, embedding_manager)
result = rag_retriever.retrieve("what is attention?")
print("Retrive document result:")
print(result)

# Simple RAG Pipeline with Groq LLM Output

In [ ]:
!pip install langchain-groq python-dotenv

In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
# from dotenv import load_dotenv # Commented out, as we'll use Colab's secrets manager
from google.colab import userdata # Import userdata for Colab secrets

# load_dotenv() # Commented out

# Get the GROQ LLM API key from Colab's Secrets Manager
groq_api_key = userdata.get("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in Colab Secrets. Please set it there.")

print("GROQ_API_KEY loaded successfully from Colab Secrets.")

# Initialize the GRQ LLM with a supported model
llm = ChatGroq(model_name="openai/gpt-oss-120b", groq_api_key=groq_api_key, temperature=0.1)

# Simple RAG function: retrieval context + generate response
def rag_pipeline(query: str, retriver, llm, top_k: int = 5):
  print(f"RAG pipeline with query: '{query}'")

  # retrieve the context
  results = retriver.retrieve(query, top_k=top_k)

  context = "\n".join([doc['page_content'] for doc in results])
  if not context:
    return "No relevant context found to answer the question"

  # generate response using GROQ LLM
  prompt=f"""Use the following pieces of context to answer the question concisely.
  Context:
  {context}
  Question: {query}
  Answer:"""
  response = llm.invoke([prompt.format(context=context, question = query)])
  return response.content


In [ ]:
answer = rag_pipeline("What is Attention?", rag_retriever, llm)
#answer = rag_pipeline("Limitation of Attention?", rag_retriever, llm)
print(answer)

# Enhance RAG Pipeline

In [ ]:
# Enhanced RAG Pipeline Feature
def rag_advanced_pipeline(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
  """
  RAG pipeline with advanced features:
  - Returns answer, sources, confidence score and optionally full context
  - Context filtering based on similarity score
  """
  results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
  if not results:
    return {'answer':'No relevant context found to answer the question', 'source': [], 'confidence': 0.0, 'context': ''}

  # prepare the context and source
  context = "\n".join([doc['page_content'] for doc in results])
  sources = [{
      'source': doc['metadata']['source_file'],
      'page': doc['metadata'].get('page', 'Unknown'),
      'similarity_score': doc['similarity_score'],
      'distance': doc['distance'],
      'rank': doc['rank'],
      'preview': doc['page_content'][:200] + '...'
  } for doc in results]
  confidence = max([doc['similarity_score'] for doc in results])

  # generate response using GROQ LLM
  prompt=f"""Use the following pieces of context to answer the question concisely.
  Context:
  {context}
  Question: {query}
  Answer:"""
  response = llm.invoke([prompt.format(context=context, question = query)])
  answer = response.content

  if return_context:
    return {'answer': answer, 'source': sources, 'confidence': confidence, 'context': context}
  else:
    return {'answer': answer, 'source': sources, 'confidence': confidence}



In [ ]:
# Example
#query = "What is Attention?"
query = "Overview of Attention Mechanism Architecture?"
#query = "What is Petrol Engine?"
result = rag_advanced_pipeline(query, rag_retriever, llm, top_k=5, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['source'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:200])